In [1]:
%load_ext sql
%sql --section duck
import epyfun
sqlite_file = epyfun.get_latest_file("./data/")

Loading configurations from C:\OneDrive\All\strom\pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaycon,False
dsn_filename,./connections.ini


Connecting to 'duck'

In [ ]:
%%sql 

SELECT 
    EXTRACT(YEAR FROM minute) AS year,    
    EXTRACT(MONTH FROM minute) AS month,
    1.0 * AVG(cm) AS cmm,
    1.0 * SUM(cm) AS tot,
    24.0 * 60.0 * cmm AS wd,
    SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS wobs,
FROM waermestrom_minute
GROUP BY ALL
ORDER BY year, month
;

In [ ]:
%%sql 

x << SELECT *
FROM waermestrom_minute
WHERE minute >= '2024-05-07'
;

In [ ]:
%%sql

SELECT * 
FROM normalstrom
ORDER BY date
;


In [ ]:
%%sql

CREATE OR REPLACE TABLE strom AS
WITH strom_sqlite AS (
    SELECT 
        meterid, 
        -- Blob Functions, because most columns get read as blob
        -- https://duckdb.org/docs/sql/functions/blob
        CAST(decode(date) AS DATETIME) AS date, 
        CAST(decode(value) AS INT) AS value,
        CAST(decode(first) AS INT) AS first
    FROM sqlite_scan('{{sqlite_file}}', 'reading') 
    WHERE meterid = 1 OR meterid = 2 OR meterid = 3
)
SELECT 
    *,
    date_sub(
        'minute', 
        lag(date, 1) OVER(PARTITION BY meterid ORDER BY date),
        date
    ) AS minutes, 
    -- use (1/(1-first)) to induce NA when it is first measurement
    value * (1/(1-first)) - lag(value, 1) OVER(PARTITION BY meterid ORDER BY date) AS consumption,
    1.0 * consumption / minutes AS cm
FROM strom_sqlite
ORDER BY date
;


In [ ]:
%%sql

CREATE OR REPLACE TABLE strom_minute AS
WITH minutes_table AS (
  SELECT UNNEST(generate_series(ts[1], ts[2], interval 1 minute)) AS minute, generate_series AS meterid
  FROM (VALUES (
    [(SELECT MIN(date) FROM strom), (SELECT MAX(date) FROM strom)]
  )) t(ts), generate_series(1, 3)
)
SELECT 
  minutes_table.meterid,
  minutes_table.minute,
  FIRST_VALUE(strom.cm IGNORE NULLS) OVER(
    PARTITION BY minutes_table.meterid 
    ORDER BY minutes_table.minute 
    ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
  ) AS cm,
  strom.date,
  strom.value,
  strom.minutes,
  strom.consumption 
FROM minutes_table
LEFT JOIN strom
ON minutes_table.minute = strom.date AND minutes_table.meterid = strom.meterid
ORDER BY minutes_table.minute
;


In [3]:
%%sql

SELECT 
  date,
  "1_cd" AS nd,
  "2_cd" + "3_cd" AS wd,
  "2_cd" AS nt,
  "3_cd" AS ht,
  GREATEST("1_obs", "2_obs", "3_obs") AS obs
FROM (
  WITH cte AS (
    SELECT 
      meterid,
      minute::DATE AS date,
      SUM(cm) AS tot,
      24.0 * 60.0 * AVG(cm) AS cd,
      SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS obs
    FROM strom_minute
    --WHERE meterid = 1
    GROUP BY ALL
    --HAVING round(tot, 7) != round(cd, 7)
    ORDER BY date
  )
  PIVOT_WIDER cte
  ON meterid
  USING first(cd) AS cd, first(obs) AS obs
  GROUP BY date
)
;


,nd,wd,nt,ht,obs,date,1_cd,1_obs,2_cd,2_obs,3_cd,3_obs
0,6.048074,10.187794,4.874659,5.313135,2.0,2016-07-04,6.048074,2.0,4.874659,2.0,5.313135,2.0
1,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-05,4.465727,0.0,3.278852,0.0,3.722357,0.0
2,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-06,4.465727,0.0,3.278852,0.0,3.722357,0.0
3,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-07,4.465727,0.0,3.278852,0.0,3.722357,0.0
4,4.465727,7.001210,3.278852,3.722357,0.0,2016-07-08,4.465727,0.0,3.278852,0.0,3.722357,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2881,10.919332,6.546488,2.546488,4.000000,2.0,2024-05-24,10.919332,2.0,2.546488,2.0,4.000000,2.0
2882,10.313539,6.011990,6.011990,0.000000,1.0,2024-05-25,10.313539,1.0,6.011990,1.0,0.000000,1.0
2883,13.428571,7.425847,7.425847,0.000000,4.0,2024-05-26,13.428571,4.0,7.425847,4.0,0.000000,4.0
2884,7.178344,5.044586,2.000000,3.044586,4.0,2024-05-27,7.178344,4.0,2.000000,4.0,3.044586,4.0


In [22]:
%%sql

  WITH cte AS (
    SELECT CAST(minute AS DATE) AS date, * FROM strom_minute
  )
  PIVOT_WIDER cte
  ON meterid
  USING 
    SUM(cm) AS tot,
    AVG(cm* 24.0 * 60.0)  AS cd, 
    SUM(CASE WHEN value IS NOT NULL THEN 1 ELSE 0 END) AS obs
  GROUP BY date

;


,date,1_tot,1_cd,1_obs,2_tot,2_cd,2_obs,3_tot,3_cd,3_obs
0,2016-07-04,4.074050,6.048074,2.0,3.283625,4.874659,2.0,3.578986,5.313135,2.0
1,2016-07-05,4.465727,4.465727,0.0,3.278852,3.278852,0.0,3.722357,3.722357,0.0
2,2016-07-06,4.465727,4.465727,0.0,3.278852,3.278852,0.0,3.722357,3.722357,0.0
3,2016-07-07,4.465727,4.465727,0.0,3.278852,3.278852,0.0,3.722357,3.722357,0.0
4,2016-07-08,4.465727,4.465727,0.0,3.278852,3.278852,0.0,3.722357,3.722357,0.0
...,...,...,...,...,...,...,...,...,...,...
2881,2024-05-24,10.919332,10.919332,2.0,2.546488,2.546488,2.0,4.000000,4.000000,2.0
2882,2024-05-25,10.313539,10.313539,1.0,6.011990,6.011990,1.0,0.000000,0.000000,1.0
2883,2024-05-26,13.428571,13.428571,4.0,7.425847,7.425847,4.0,0.000000,0.000000,4.0
2884,2024-05-27,7.178344,7.178344,4.0,2.000000,2.000000,4.0,3.044586,3.044586,4.0
